In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/MyDrive

Mounted at /content/gdrive
/content/gdrive/MyDrive


In [ ]:
import os
import time
import math
import torch
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from numpy.random import seed
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import torch.nn.functional as f
from plotly.offline import iplot
from torch.autograd import Variable
from sklearn.metrics import r2_score
from pandas import DataFrame, concat, read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import make_classification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.nn.utils.parametrizations import weight_norm
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler

seed(111)

In [ ]:
data_path = "/content/gdrive/MyDrive/KFUPM/SolarRadiationPrediction/RI-Hourly-Met-Data-2012-2020.xlsx"
data = pd.read_excel(data_path, skiprows=[1])
data.head(10)

,TIMESTAMP,GSR,BP,Temp,RH,WS,WD,RF
0,2012-01-01 00:00:00,0.00,1006,18.19,56.21,2.779,110.3,0.0
1,2012-01-01 01:00:00,0.00,1005,17.86,62.98,3.181,114.3,0.0
2,2012-01-01 02:00:00,0.00,1005,17.19,66.97,3.869,127.1,0.0
3,2012-01-01 03:00:00,0.00,1005,16.77,72.50,3.798,138.8,0.0
4,2012-01-01 04:00:00,0.00,1004,16.31,79.58,3.754,149.9,0.0
5,2012-01-01 05:00:00,0.00,1004,15.68,85.90,3.718,165.0,0.0
6,2012-01-01 06:00:00,0.00,1004,15.30,89.00,4.270,173.5,0.0
7,2012-01-01 07:00:00,16.02,1005,13.53,86.20,4.422,215.0,0.0
8,2012-01-01 08:00:00,162.30,1006,13.99,69.14,4.421,215.7,0.0
9,2012-01-01 09:00:00,354.30,1006,15.42,63.28,4.744,190.8,0.0


In [ ]:
# dataset = read_csv("/content/gdrive/MyDrive/KFUPM/SolarRadiationPrediction/pollution.csv", header=0, index_col=0)
data = data.drop_duplicates(subset=['TIMESTAMP'], keep='last')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75133 entries, 0 to 83893
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   TIMESTAMP  75133 non-null  datetime64[ns]
 1   GSR        75133 non-null  float64       
 2   BP         75133 non-null  int64         
 3   Temp       75133 non-null  float64       
 4   RH         75133 non-null  float64       
 5   WS         75109 non-null  float64       
 6   WD         75109 non-null  float64       
 7   RF         66373 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 5.2 MB


In [ ]:
data_copy = data.copy()
data_copy = data_copy.set_index(['TIMESTAMP'])
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75133 entries, 2012-01-01 00:00:00 to 2020-12-31 23:00:00
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GSR     75133 non-null  float64
 1   BP      75133 non-null  int64  
 2   Temp    75133 non-null  float64
 3   RH      75133 non-null  float64
 4   WS      75109 non-null  float64
 5   WD      75109 non-null  float64
 6   RF      66373 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 4.6 MB


In [ ]:
data_copy = data_copy[['GSR','BP', 'Temp', 'RH', 'WS', 'WD']]
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75133 entries, 2012-01-01 00:00:00 to 2020-12-31 23:00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GSR     75133 non-null  float64
 1   BP      75133 non-null  int64  
 2   Temp    75133 non-null  float64
 3   RH      75133 non-null  float64
 4   WS      75109 non-null  float64
 5   WD      75109 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 4.0 MB


In [ ]:
df_features = (
                data_copy
                .assign(hour = data_copy.index.hour)
                .assign(day_of_year = data_copy.index.day_of_year)
              )
df_features.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75133 entries, 2012-01-01 00:00:00 to 2020-12-31 23:00:00
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GSR          75133 non-null  float64
 1   BP           75133 non-null  int64  
 2   Temp         75133 non-null  float64
 3   RH           75133 non-null  float64
 4   WS           75109 non-null  float64
 5   WD           75109 non-null  float64
 6   hour         75133 non-null  int32  
 7   day_of_year  75133 non-null  int32  
dtypes: float64(5), int32(2), int64(1)
memory usage: 4.6 MB


In [ ]:
def generate_cyclical_features(df, col_name, period, start_num=0):
    kwargs = {
        f'sin_{col_name}' : lambda x: np.sin(2*np.pi*(df[col_name]-start_num)/period),
        # f'cos_{col_name}' : lambda x: np.cos(2*np.pi*(df[col_name]-start_num)/period)
             }
    return df.assign(**kwargs)#.drop(columns=[col_name])

df_features = generate_cyclical_features(df_features, 'hour', 24*2, 0)
df_features = generate_cyclical_features(df_features, 'day_of_year', 365*2, 0)

In [ ]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75133 entries, 2012-01-01 00:00:00 to 2020-12-31 23:00:00
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   GSR              75133 non-null  float64
 1   BP               75133 non-null  int64  
 2   Temp             75133 non-null  float64
 3   RH               75133 non-null  float64
 4   WS               75109 non-null  float64
 5   WD               75109 non-null  float64
 6   hour             75133 non-null  int32  
 7   day_of_year      75133 non-null  int32  
 8   sin_hour         75133 non-null  float64
 9   sin_day_of_year  75133 non-null  float64
dtypes: float64(7), int32(2), int64(1)
memory usage: 5.7 MB


In [ ]:
values = df_features.values

In [ ]:
values = values.astype('float32')

In [ ]:
Tx = 48  # {12, 24, 48, 72}
Ty = 60  # {1, 6, 12, 24, 36, 48, 60}

inp_var = len(df_features.columns)  # Number of input variables
out_var = 1  # Number of output variables
train_split = 0.7  # Size of the training set
val_split = 0.15  # Size of the validation set

In [ ]:
# Convert series to supervised learning setup
def series_to_supervised(data, n_in=1, n_out=1, drop_nan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i_ in range(n_in, 0, -1):
        cols.append(df.shift(i_))
        names += [('var%d(t-%d)' % (j + 1, i_)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for _i in range(0, n_out):
        cols.append(df.shift(-_i))
        if _i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, _i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if drop_nan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
df_features.head(10)

,GSR,BP,Temp,RH,WS,WD,hour,day_of_year,sin_hour,sin_day_of_year
TIMESTAMP,,,,,,,,,,
2012-01-01 00:00:00,0.00,1006,18.19,56.21,2.779,110.3,0,1,0.000000,0.008607
2012-01-01 01:00:00,0.00,1005,17.86,62.98,3.181,114.3,1,1,0.130526,0.008607
2012-01-01 02:00:00,0.00,1005,17.19,66.97,3.869,127.1,2,1,0.258819,0.008607
2012-01-01 03:00:00,0.00,1005,16.77,72.50,3.798,138.8,3,1,0.382683,0.008607
2012-01-01 04:00:00,0.00,1004,16.31,79.58,3.754,149.9,4,1,0.500000,0.008607
2012-01-01 05:00:00,0.00,1004,15.68,85.90,3.718,165.0,5,1,0.608761,0.008607
2012-01-01 06:00:00,0.00,1004,15.30,89.00,4.270,173.5,6,1,0.707107,0.008607
2012-01-01 07:00:00,16.02,1005,13.53,86.20,4.422,215.0,7,1,0.793353,0.008607
2012-01-01 08:00:00,162.30,1006,13.99,69.14,4.421,215.7,8,1,0.866025,0.008607


In [ ]:
reframed = series_to_supervised(df_features, Tx, Ty)
reframed_ = reframed.values
reframed.head(10)

,var1(t-48),var2(t-48),var3(t-48),var4(t-48),var5(t-48),var6(t-48),var7(t-48),var8(t-48),var9(t-48),var10(t-48),...,var1(t+59),var2(t+59),var3(t+59),var4(t+59),var5(t+59),var6(t+59),var7(t+59),var8(t+59),var9(t+59),var10(t+59)
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2012-01-03 00:00:00,0.00,1006.0,18.19,56.21,2.779,110.3,0.0,1.0,0.000000,0.008607,...,634.100,1007.0,16.71,52.37,4.589,323.400,11.0,5.0,0.991445,0.043022
2012-01-03 01:00:00,0.00,1005.0,17.86,62.98,3.181,114.3,1.0,1.0,0.130526,0.008607,...,661.800,1006.0,17.79,51.95,4.800,344.000,12.0,5.0,1.000000,0.043022
2012-01-03 02:00:00,0.00,1005.0,17.19,66.97,3.869,127.1,2.0,1.0,0.258819,0.008607,...,605.200,1005.0,18.31,51.41,4.424,350.300,13.0,5.0,0.991445,0.043022
2012-01-03 03:00:00,0.00,1005.0,16.77,72.50,3.798,138.8,3.0,1.0,0.382683,0.008607,...,479.200,1004.0,18.73,50.41,4.224,344.500,14.0,5.0,0.965926,0.043022
2012-01-03 04:00:00,0.00,1004.0,16.31,79.58,3.754,149.9,4.0,1.0,0.500000,0.008607,...,336.800,1004.0,18.45,52.02,4.801,5.405,15.0,5.0,0.923880,0.043022
2012-01-03 05:00:00,0.00,1004.0,15.68,85.90,3.718,165.0,5.0,1.0,0.608761,0.008607,...,204.000,1004.0,17.78,56.07,5.529,14.080,16.0,5.0,0.866025,0.043022
2012-01-03 06:00:00,0.00,1004.0,15.30,89.00,4.270,173.5,6.0,1.0,0.707107,0.008607,...,48.610,1004.0,17.14,59.16,5.059,5.461,17.0,5.0,0.793353,0.043022
2012-01-03 07:00:00,16.02,1005.0,13.53,86.20,4.422,215.0,7.0,1.0,0.793353,0.008607,...,0.157,1005.0,16.65,61.61,2.976,350.800,18.0,5.0,0.707107,0.043022
2012-01-03 08:00:00,162.30,1006.0,13.99,69.14,4.421,215.7,8.0,1.0,0.866025,0.008607,...,0.000,1005.0,16.02,64.32,2.588,331.300,19.0,5.0,0.608761,0.043022


In [ ]:
n_obs = Tx * inp_var
print(n_obs)

480


In [ ]:
X = reframed_[:, : n_obs]  # (43789, 80)
y1 = reframed_[:, n_obs:]  # (43789, 16)

In [ ]:
y1[0]

array([0.00000000e+00, 1.00100000e+03, 1.62700000e+01, 8.08000000e+01,
       4.40100000e+00, 3.02300000e+02, 0.00000000e+00, 3.00000000e+00,
       0.00000000e+00, 2.58184402e-02, 0.00000000e+00, 1.00100000e+03,
       1.59600000e+01, 8.72000000e+01, 4.66000000e+00, 2.92300000e+02,
       1.00000000e+00, 3.00000000e+00, 1.30526192e-01, 2.58184402e-02,
       0.00000000e+00, 1.00000000e+03, 1.60400000e+01, 9.16000000e+01,
       4.82700000e+00, 3.05800000e+02, 2.00000000e+00, 3.00000000e+00,
       2.58819045e-01, 2.58184402e-02, 0.00000000e+00, 1.00000000e+03,
       1.62800000e+01, 9.26000000e+01, 5.01000000e+00, 2.97000000e+02,
       3.00000000e+00, 3.00000000e+00, 3.82683432e-01, 2.58184402e-02,
       0.00000000e+00, 1.00000000e+03, 1.64900000e+01, 9.30000000e+01,
       5.15600000e+00, 2.97400000e+02, 4.00000000e+00, 3.00000000e+00,
       5.00000000e-01, 2.58184402e-02, 0.00000000e+00, 1.00100000e+03,
       1.69500000e+01, 9.31000000e+01, 6.35200000e+00, 3.06600000e+02,
      

In [ ]:
y = np.zeros((y1.shape[0], Ty))
y.shape

(74895, 60)

In [ ]:
for i in range(0, Ty):
    index = inp_var * i  # 0, 8, 16, 24, 32
    y[:, i] = y1[:, index]

In [ ]:
X = X.reshape((X.shape[0], Tx, inp_var))

In [ ]:
def seq_split(x_data, y_data, train_split_, val_split_, x_df):
    train_size = int(len(x_data) * train_split_)
    val_size = int(len(x_data) * (train_split_ + val_split_))

    x_train_ = x_data[0: train_size, :, :]
    y_train_ = y_data[0: train_size]

    train_df = x_df.iloc[0: train_size]

    x_val_ = x_data[train_size: val_size, :, :]
    y_val_ = y_data[train_size: val_size]

    val_df = x_df.iloc[train_size: val_size]

    x_test_ = x_data[val_size: len(x_data), :, :]
    y_test_ = y_data[val_size: len(y_data), :]

    test_df = x_df.iloc[val_size: len(y_data)]

    return x_train_, y_train_, x_val_, y_val_, x_test_, y_test_, test_df

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test, test_df = seq_split(X, y, train_split, val_split, reframed)
y_test_original = y_test.copy()

In [ ]:
test_df.head(10)

,var1(t-48),var2(t-48),var3(t-48),var4(t-48),var5(t-48),var6(t-48),var7(t-48),var8(t-48),var9(t-48),var10(t-48),...,var1(t+59),var2(t+59),var3(t+59),var4(t+59),var5(t+59),var6(t+59),var7(t+59),var8(t+59),var9(t+59),var10(t+59)
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2019-04-13 23:00:00,0.00,996.0,26.00,46.97,4.283,135.1,23.0,101.0,0.130526,0.763889,...,876.000,1000.0,30.38,11.220,3.362,256.30,10.0,106.0,0.965926,0.790946
2019-04-14 00:00:00,0.00,996.0,25.66,49.01,4.103,148.7,0.0,102.0,0.000000,0.769415,...,976.000,1000.0,31.86,8.910,3.885,253.70,11.0,106.0,0.991445,0.790946
2019-04-14 01:00:00,0.00,995.0,25.51,47.99,3.420,157.8,1.0,102.0,0.130526,0.769415,...,1021.000,999.0,33.13,7.383,2.825,264.30,12.0,106.0,1.000000,0.790946
2019-04-14 02:00:00,0.00,995.0,25.10,50.05,4.062,180.5,2.0,102.0,0.258819,0.769415,...,987.000,998.0,33.65,8.670,1.576,165.10,13.0,106.0,0.991445,0.790946
2019-04-14 03:00:00,0.00,994.0,24.88,47.46,4.914,212.3,3.0,102.0,0.382683,0.769415,...,884.000,998.0,32.43,11.940,3.296,81.90,14.0,106.0,0.965926,0.790946
2019-04-14 04:00:00,0.00,994.0,24.64,45.36,5.317,223.4,4.0,102.0,0.500000,0.769415,...,537.400,997.0,31.85,10.940,3.740,93.10,15.0,106.0,0.923880,0.790946
2019-04-14 05:00:00,0.00,995.0,24.34,44.39,4.160,239.1,5.0,102.0,0.608761,0.769415,...,258.400,997.0,31.17,11.420,3.334,93.20,16.0,106.0,0.866025,0.790946
2019-04-14 06:00:00,14.56,996.0,24.06,41.56,4.607,228.1,6.0,102.0,0.707107,0.769415,...,160.000,997.0,30.68,11.800,3.227,72.55,17.0,106.0,0.793353,0.790946
2019-04-14 07:00:00,180.40,996.0,24.21,41.44,3.592,221.8,7.0,102.0,0.793353,0.769415,...,41.830,997.0,29.31,14.930,3.360,68.03,18.0,106.0,0.707107,0.790946


In [ ]:
def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

scalar_x = get_scaler('robust')
scalar_y = get_scaler('robust')

In [ ]:
x_train_reshaped = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2]))

In [ ]:
scalar_x = scalar_x.fit(x_train_reshaped)
scalar_y = scalar_y.fit(y_train)

In [ ]:
def scale_features(x_data, y_data):
    x_data = x_data.reshape((x_data.shape[0], x_data.shape[1] * x_data.shape[2]))
    x_data = scalar_x.transform(x_data)
    x_data = x_data.reshape((x_data.shape[0], Tx, inp_var))

    y_data = scalar_y.transform(y_data)

    return x_data, y_data

In [ ]:
x_train, y_train = scale_features(x_train, y_train)
x_val, y_val = scale_features(x_val, y_val)
x_test, y_test = scale_features(x_test, y_test)

In [ ]:
print('x_train shape:', x_train.shape, 'y_train shape:', y_train.shape)  # (30652, 10, 8), (30652, 2)
print('x_val shape:', x_val.shape, 'y_val shape:', y_val.shape)     # (8758, 10, 8), (8758, 2)
print('x_test shape:', x_test.shape, 'y_test shape:', y_test.shape)

x_train shape: (52426, 48, 10) y_train shape: (52426, 60)
x_val shape: (11234, 48, 10) y_val shape: (11234, 60)
x_test shape: (11235, 48, 10) y_test shape: (11235, 60)


In [ ]:
y_test[0]

array([-0.0242743 , -0.02428404, -0.02429024, -0.02428905, -0.02431827,
       -0.02435367, -0.02437314,  0.01494717,  0.37718487,  0.89387994,
        1.35331808,  1.70897317,  1.93909233,  2.01700429,  1.94698807,
        1.77194623,  1.44367816,  1.02599844,  0.5504773 ,  0.13017728,
       -0.02290863, -0.02423534, -0.02423534, -0.02426456, -0.0242743 ,
       -0.02427076, -0.02427931, -0.02428905, -0.02428551, -0.02431353,
       -0.02435248,  0.03650438,  0.44437196,  0.94565725,  1.38542356,
        1.71873996,  1.95077912,  2.05207441,  1.99959094,  1.8109098 ,
        1.48537892,  1.02368985,  0.5585038 ,  0.13302162, -0.02306448,
       -0.02419638, -0.02419638, -0.02423534, -0.02426456, -0.0242743 ,
       -0.02427076, -0.02427931, -0.02428905, -0.02428551, -0.02431472,
        0.02766444,  0.398627  ,  0.90149472,  1.33797166,  1.68198286])

In [ ]:
y_test_original[0]

array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 2.020e+01, 2.062e+02, 4.715e+02, 7.074e+02, 8.900e+02,
       1.008e+03, 1.048e+03, 1.012e+03, 9.220e+02, 7.535e+02, 5.391e+02,
       2.950e+02, 7.926e+01, 6.810e-01, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 3.126e+01, 2.407e+02, 4.981e+02, 7.239e+02, 8.950e+02,
       1.014e+03, 1.066e+03, 1.039e+03, 9.420e+02, 7.749e+02, 5.379e+02,
       2.991e+02, 8.070e+01, 5.810e-01, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 2.671e+01, 2.172e+02, 4.754e+02, 6.995e+02, 8.760e+02])

In [ ]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

In [ ]:
class AttentionBlock(nn.Module):
    def __init__(self, in_channels, key_size, value_size):
        super(AttentionBlock, self).__init__()
        self.linear_query = nn.Linear(in_channels, key_size)
        self.linear_keys = nn.Linear(in_channels, key_size)
        self.linear_values = nn.Linear(in_channels, value_size)
        self.sqrt_key_size = math.sqrt(key_size)

    def forward(self, input_):
        # input is dim (N, in_channels, T) where N is the batch_size, and T is the sequence length
        mask = np.array([[1 if i > j else 0 for i in range(input_.size(2))] for j in range(input_.size(2))])
        if input_.is_cuda:
            mask = torch.BoolTensor(mask).cuda(input_.get_device())
        else:
            mask = torch.BoolTensor(mask)
        # mask = mask.bool()

        input_ = input_.permute(0, 2, 1)  # input: [N, T, inchannels]
        keys = self.linear_keys(input_)  # keys: (N, T, key_size)
        query = self.linear_query(input_)  # query: (N, T, key_size)
        values = self.linear_values(input_)  # values: (N, T, value_size)
        temp = torch.bmm(query, torch.transpose(keys, 1, 2))  # shape: (N, T, T)
        temp.data.masked_fill_(mask, -float('inf'))

        weight_temp = torch.nn.functional.softmax(temp / self.sqrt_key_size, dim=1)
        # temp: (N, T, T), broadcasting over any slice [:, x, :], each row of the matrix
        # weight_temp_vert = F.softmax(temp / self.sqrt_key_size, dim=1)
        # temp: (N, T, T), broadcasting over any slice [:, x, :], each row of the matrix
        # weight_temp_hori = F.softmax(temp / self.sqrt_key_size, dim=2)
        # temp: (N, T, T), broadcasting over any slice [:, x, :], each row of the matrix
        # weight_temp = (weight_temp_hori + weight_temp_vert)/2
        value_attentioned = torch.bmm(weight_temp, values).permute(0, 2, 1)  # shape: (N, T, value_size)

        return value_attentioned, weight_temp  # value_attentioned: [N, in_channels, T], weight_temp: [N, T, T]

In [ ]:
class ChannelSELayer1d(nn.Module):
    def __init__(self, num_channels, reduction_ratio=4):
        """

        :param num_channels: No of input channels
        :param reduction_ratio: By how much should the num_channels should be reduced
        """
        super(ChannelSELayer1d, self).__init__()
        num_channels_reduced = num_channels // reduction_ratio
        self.reduction_ratio = reduction_ratio
        self.fc1 = nn.Linear(num_channels, num_channels_reduced, bias=True)
        self.fc2 = nn.Linear(num_channels_reduced, num_channels, bias=True)
        self.activ_1 = nn.ReLU()
        self.activ_2 = nn.Sigmoid()

    def forward(self, input_tensor):
        """

        :param input_tensor: X, shape = (batch_size, num_channels, H)
        :return: output tensor
        """
        batch_size, num_channels, H = input_tensor.size()
        # Average along each channel
        squeeze_tensor = input_tensor.view(batch_size, num_channels, -1).mean(dim=2)

        # channel excitation
        fc_out_1 = self.activ_1(self.fc1(squeeze_tensor))
        fc_out_2 = self.activ_2(self.fc2(fc_out_1))

        # a, b = squeeze_tensor.size()
        # output_tensor = torch.mul(input_tensor, fc_out_2.view(batch_size, num_channels, 1))
        return input_tensor * fc_out_2.view(batch_size, num_channels, 1)


class SpatialSELayer1d(nn.Module):

    def __init__(self, num_channels):
        """

        :param num_channels: No of input channels
        """
        super(SpatialSELayer1d, self).__init__()
        self.conv = nn.Conv1d(num_channels, 1, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor, weights=None):
        """

        :param weights: weights for few shot learning
        :param input_tensor: X, shape = (batch_size, num_channels, W)
        :return: output_tensor
        """
        # spatial squeeze
        batch_size, channel, a = input_tensor.size()

        if weights is not None:
            weights = torch.mean(weights, dim=0)
            weights = weights.view(1, channel, 1)
            out = f.conv2d(input_tensor, weights)
        else:
            out = self.conv(input_tensor)
        squeeze_tensor = self.sigmoid(out)

        return input_tensor * squeeze_tensor.view(batch_size, 1, a)


class ChannelSpatialSELayer1d(nn.Module):

    def __init__(self, num_channels, reduction_ratio=2):
        """

        :param num_channels: No of input channels
        :param reduction_ratio: By how much should the num_channels should be reduced
        """
        super(ChannelSpatialSELayer1d, self).__init__()
        self.cSE = ChannelSELayer1d(num_channels, reduction_ratio)
        self.sSE = SpatialSELayer1d(num_channels)

    def forward(self, input_tensor):
        """

        :param input_tensor: X, shape = (batch_size, num_channels, W)
        :return: output_tensor
        """

        return torch.max(self.cSE(input_tensor), self.sSE(input_tensor))

In [ ]:
def _make_layers(num_sub_blocks, n_inputs, n_outputs, kernel_size, stride, dilation,
                 padding, vhdrop_layer, dropout=0.2):
    layers_list = []

    if vhdrop_layer is not None:
        layers_list.append(vhdrop_layer)
    layers_list.append(
        weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation)))
    layers_list.append(Chomp1d(padding))
    layers_list.append(nn.ReLU())
    layers_list.append(nn.Dropout(dropout))
    for _ in range(num_sub_blocks - 1):
        layers_list.append(
            weight_norm(
                nn.Conv1d(n_outputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation)))
        layers_list.append(Chomp1d(padding))
        layers_list.append(nn.ReLU())
        layers_list.append(nn.Dropout(dropout))

    return nn.Sequential(*layers_list)


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, key_size, num_sub_blocks, temp_attn, en_res,
                 conv, stride, dilation, padding, vhdrop_layer, visual, squeeze, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.visual = visual
        self.en_res = en_res
        self.conv = conv
        self.temp_attn = temp_attn
        self.squeeze = squeeze
        if self.temp_attn:
            # if self.nheads > 1:
            #     self.attentions = [AttentionBlock(n_inputs, key_size, n_inputs) for _ in range(self.nheads)]
            #     for i, attention in enumerate(self.attentions):
            #         self.add_module('attention_{}'.format(i), attention)
            #     # self.cat_attentions = AttentionBlock(n_inputs * self.nheads, n_inputs, n_inputs)
            #     self.linear_cat = nn.Linear(n_inputs * self.nheads, n_inputs)
            # else:
            self.attention = AttentionBlock(n_inputs, key_size, n_inputs)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        if self.conv:
            self.net = _make_layers(num_sub_blocks, n_inputs, n_outputs, kernel_size, stride, dilation,
                                    padding, vhdrop_layer, dropout)
            self.init_weights()

        self.sq_ex = nn.Sequential(ChannelSpatialSELayer1d(n_outputs, reduction_ratio=2))

    def init_weights(self):
        layer_idx_list = []
        for name, _ in self.net.named_parameters():
            in_layer_param_list = name.split('.')
            layer_idx_list.append(int(in_layer_param_list[0]))
        layer_idxes = list(set(layer_idx_list))
        for idx in layer_idxes:
            getattr(self.net[idx], 'weight').data.normal_(0, 0.01)

        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        # x: [N, emb_size, T]
        # global attn_weight
        if self.temp_attn:
            # x = x if self.downsample is None else self.downsample(x)
            out_attn, attn_weight = self.attention(x)
            if self.conv:
                out = self.net(out_attn)
            else:
                out = out_attn

            if self.squeeze:
                out = self.sq_ex(out)
            weight_x = torch.nn.functional.softmax(attn_weight.sum(dim=2), dim=1)
            en_res_x = weight_x.unsqueeze(2).repeat(1, 1, x.size(1)).transpose(1, 2) * x
            en_res_x = en_res_x if self.downsample is None else self.downsample(en_res_x)

            res = self.downsample(x) if self.downsample else x

            if self.visual:
                attn_weight_cpu = attn_weight.detach().cpu().numpy()
            else:
                attn_weight_cpu = [0] * 10
            del attn_weight

            if self.en_res:
                return self.relu(out + res + en_res_x), attn_weight_cpu
            else:
                return self.relu(out + res), attn_weight_cpu
        else:
            out = self.net(x)
            if self.squeeze:
                out = self.sq_ex(out)
            res = x if self.downsample is None else self.downsample(x)
            return self.relu(out + res)  # return: [N, emb_size, T]

In [ ]:
class TemporalConvNet(nn.Module):
    def __init__(self, emb_size, num_channels, num_sub_blocks, temp_attn, en_res,
                 conv, key_size, kernel_size, visual, squeeze, vhdropout=0.0, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        self.vhdrop_layer = None
        # layers.append(nn.Conv1d(emb_size*2, emb_size, 1))
        self.temp_attn = temp_attn
        # self.temp_attn_share = AttentionBlock(emb_size, key_size, emb_size)
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = emb_size if i == 0 else num_channels[i - 1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, key_size, num_sub_blocks, temp_attn,
                                     en_res, conv, stride=1, dilation=dilation_size,
                                     padding=(kernel_size - 1) * dilation_size, vhdrop_layer=self.vhdrop_layer,
                                     visual=visual, squeeze = squeeze, dropout=dropout)]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        # x: [batch_size, seq_len, emb_size]
        attn_weight_list = []
        if self.temp_attn:
            out = x
            for i in range(len(self.network)):
                out, attn_weight_ = self.network[i](out)
                # print("the len of attn_weight", len(attn_weight))
                # if len(attn_weight) == 64:
                #     attn_weight_list.append([attn_weight[18], attn_weight[19]])
                attn_weight_list.append([attn_weight_[0], attn_weight_[-1]])
            return out, attn_weight_list
        else:
            return self.network(x)

In [ ]:
class TCANet(nn.Module):

    def __init__(self, Ty, emb_size, input_size, emb_layers, num_channels, decoder_hidden, num_sub_blocks, temp_attn,
                 en_res, conv, key_size, key_size_1, kernel_size=2, dropout=0.3, emb_dropout=0.1, visual=True, squeeze=True, embed = True):
        super(TCANet, self).__init__()
        self.temp_attn = temp_attn
        self.num_levels = len(num_channels)
        self.emb_layers = emb_layers
        self.emb_size = emb_size
        self.decoder_hidden = decoder_hidden
        self.embed = embed


        self.encoder = nn.LSTM(input_size, emb_size, emb_layers, batch_first=True, dropout=emb_dropout)
        # self.encoder = nn.RNN(input_size, emb_size, emb_layers, batch_first=True, dropout=emb_dropout)
        # self.encoder = nn.Embedding(input_size, emb_size)  # replace with LSTM embedding of the time series
        # just for temporal part of the network
        if self.embed:
            self.tcanet = TemporalConvNet(emb_size, num_channels, num_sub_blocks, temp_attn, en_res, conv, key_size[1],
                                      kernel_size, visual=visual, squeeze=squeeze, dropout=dropout)
            self.compress = nn.Conv1d(key_size[1], 1, 1)
        else:
            self.tcanet = TemporalConvNet(input_size, num_channels, num_sub_blocks, temp_attn, en_res, conv, key_size[0],
                                      kernel_size, visual=visual, squeeze=squeeze, dropout=dropout)
            self.compress = nn.Conv1d(key_size[0], 1, 1)

        self.tcanet_spatial = TemporalConvNet(emb_size, num_channels, num_sub_blocks, temp_attn, en_res, conv, key_size[1],
                                      kernel_size, visual=visual, squeeze=squeeze, dropout=dropout)
        # self.tcanet = WeightDropout(self.tcanet, self.get_conv_names(num_channels), wdrop)
        # self.drop = VariationalDropout(emb_dropout) # drop some embeded features, e.g. [16,80,600]->[16,80,421]
        self.decoder_lstm = nn.LSTM(num_channels[-1], decoder_hidden, emb_layers, batch_first=True,
                                    dropout=emb_dropout)
        self.compress_spatial = nn.Conv1d(key_size_1, 1, 1)
        self.relu = nn.ReLU()
        self.decoder = nn.Linear(decoder_hidden*2, 1)  # decoder should be an LSTM before a linear
        self.init_weights()
        self.Ty = Ty # TODO: Fix this area

    def init_weights(self):
        # self.encoder.hidden[0].data.normal_(0, 0.01)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.normal_(0, 0.01)

    def forward(self, input_, spatial_input):
        """Input ought to have dimension (N, C_in, L_in), where L_in is the seq_len; here the input is (N, L, C)"""
        # input: [batchsize, seq_len]


        if self.embed:
            emb, (_h, _c) = self.encoder(input_)
        else:
            emb = input_
            # emb = emb.transpose(1, 2)

        # h0 = torch.zeros(self.emb_layers, input_.size(0), self.emb_size).requires_grad_()
        # emb, h0 = self.encoder(input_, h0.detach())
        batch_size = input_.size()[0]
        raw_pred_seq = torch.empty((batch_size, 0), dtype=torch.float32)

        if self.temp_attn:
            y, attn_weight_list = self.tcanet(emb.transpose(1, 2))
            y_spatial, attn_weight_list_spatial = self.tcanet_spatial(spatial_input.transpose(1, 2))
            temporal_input = y.transpose(1, 2)
            spatial_input = y_spatial.transpose(1, 2)
            # y = self.compress(y)
            # y = self.relu(y)
            # y_spatial = self.compress_spatial(y_spatial)            # y_spatial = self.relu(y_spatial)

            __h = torch.zeros(self.emb_layers, input_.size(0), self.decoder_hidden).requires_grad_().to(device)

            __c = torch.zeros(self.emb_layers, input_.size(0), self.decoder_hidden).requires_grad_().to(device)

            ___h = torch.zeros(self.emb_layers, input_.size(0), self.decoder_hidden).requires_grad_().to(device)

            ___c = torch.zeros(self.emb_layers, input_.size(0), self.decoder_hidden).requires_grad_().to(device)

            for i in range(self.Ty):
                y, (__h, __c) = self.decoder_lstm(temporal_input, (__h, __c))
                y_spatial, (___h, ___c) = self.decoder_lstm(spatial_input, (___h, ___c))

                y = y[:, -1, :]
                y_spatial = y_spatial[:, -1, :]

                y_new = torch.cat([y, y_spatial], dim=-1)
                # y_new = torch.add(y, y_spatial)
                y_new = self.decoder(y_new)
                raw_pred_seq = torch.cat((raw_pred_seq, y_new.cpu()), 1)
            return raw_pred_seq.contiguous() # , [attn_weight_list[0], attn_weight_list[self.num_levels // 2], attn_weight_list[-1]]
        else:
            y = self.tcanet(emb.transpose(1, 2))
            y_spatial = self.tcanet(spatial_input.transpose(1, 2))
            y = y.transpose(1, 2)
            y_spatial = y_spatial.transpose(1, 2)
            # y = self.compress(y)
            # y_spatial = self.compress_spatial(y_spatial)
            y, (__h, __c) = self.decoder_lstm(y)
            y_spatial, (___h, ___c) = self.decoder_lstm(y_spatial)

            y = y[:, -1, :]
            y_spatial = y_spatial[:, -1, :]

            y_new = torch.cat([y, y_spatial], dim=-1)
            y_new = self.decoder(y_new)
            return y_new.contiguous()

In [ ]:
# emb_size = 5
# num_channels = [16]*(4-1) + [16]
# num_sub_blocks = 2
# temp_attn = True
# en_res = True
# conv = True
# key_size = 5
# key_size_1 = 8
# kernel_size = 3
# visual = False
# emb_layers = 2
# input_size = 8
# decoder_hidden = 8

In [ ]:
emb_size = Tx
num_channels = [64]*(6-1) + [64] # [64]*(6-1) + [64] # [128]*(4-1) + [128]
num_sub_blocks = 2
temp_attn = True
en_res = True
conv = True
key_size_1 = inp_var
kernel_size = 3
visual = False
emb_layers = 3
input_size = inp_var
decoder_hidden = 128
squeeze__ = False
embed = False
key_size = [inp_var, Tx]

In [ ]:
model_tcn = TCANet(Ty, emb_size, input_size, emb_layers, num_channels,
                   decoder_hidden, num_sub_blocks, temp_attn, en_res,
                   conv, key_size, key_size_1, kernel_size=2, dropout=0.3,
                   emb_dropout=0.1, visual=False, squeeze=squeeze__, embed=embed)

In [ ]:
cuda = torch.cuda.is_available()
num_workers = 2 if cuda else 0
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))

Cuda = True with num_workers = 2


In [ ]:
device = torch.device("cuda" if cuda else "cpu")
model_tcn.to(device)
print("done")

done


In [ ]:
layer_dim = 3
dropout = 0.2
n_epochs = 100
context_dim = 4
learning_rate = 1e-3
weight_decay = 1e-6
loss_fn = nn.MSELoss() # reduction="mean"
optimizer = optim.Adam(model_tcn.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, threshold=0.01)

In [ ]:
pers_x_input = torch.ones(2, Tx, 10) # 26277, 5, 8
pers_s_input = torch.ones(2, 10, Tx) # 26277, 5, 8
pers_yhat0_train_input = torch.ones(2, 6)
out_test = model_tcn(pers_x_input.to(device), pers_s_input.to(device))
print(out_test.size())
# print(out_test[0])

torch.Size([2, 60])


In [ ]:
class MyDataset(Dataset):

    # load the dataset
    def __init__(self, x_data, y_data):
        self.X = x_data
        self.S = x_data.transpose(0, 2, 1)
        self.y = y_data

    # number of rows in the dataset
    def __len__(self):
        return len(self.y)

    # get a row at an index
    def __getitem__(self, index):
        xx = torch.from_numpy(self.X[index]).float()
        s_xx = torch.from_numpy(self.S[index]).float()
        yy = torch.from_numpy(np.reshape(self.y[index], (1,Ty))).float()
        return xx, s_xx, yy #

In [ ]:
train_data_1 = MyDataset(x_train, y_train)
train_args = dict(shuffle = True, batch_size = 256,
                  num_workers = num_workers, pin_memory = True, drop_last=True) if cuda else dict(shuffle = True, batch_size = 256)
train_loader = DataLoader(train_data_1, **train_args)

# Validation dataloader
val_data = MyDataset(x_val, y_val)
val_args = dict(shuffle = False, batch_size = 256,
                num_workers = num_workers, pin_memory = True) if cuda else dict(shuffle = False, batch_size = 256)
val_loader = DataLoader(val_data, **val_args)

In [ ]:
train_data_1.__getitem__(0)[2].size()

torch.Size([1, 60])

In [ ]:
# Train the model

def train_model(train_loader, model, criterion, optimizer):
    # print("Training...")

    # Set model in 'Training mode'
    model.train()

    training_loss = 0.0
    total_predictions = 0.0
    correct_prediction = 0.0

    start_time = time.time()

    # enumerate mini batches
    for i, (data, s_data, target) in enumerate(train_loader):

        # clear the gradients
        optimizer.zero_grad()

        data = data.to(device)
        s_data = s_data.to(device)
        target = target.to(device)

        # compute the model output
        outputs = model(data, s_data)
        outputs = outputs.to(device)

        outputs = torch.reshape(outputs, target.shape)

        # calculate loss
        loss = criterion(outputs, target)

        # Backward pass
        loss.backward()

        # Update model weights
        optimizer.step()

        training_loss += loss.item()#*data.size(0)


    end_time = time.time()
    print(training_loss)
    training_loss /= len(train_loader)

    # print("Training Loss = ", training_loss, " Time = ", end_time - start_time, "s")
    # acc = (correct_prediction/total_predictions) * 100
    # print("Training accuracy = ", acc, "%")

    return training_loss, (end_time - start_time)

In [ ]:
def evaluate_model(val_loader, model, criterion):

    # print("Validating...")

    with torch.no_grad():

        running_loss = 0.0
        total_predictions = 0.0
        correct_prediction = 0.0

        # Set model in validation mode
        model.eval()

        for i, (inputs, s_inputs, target) in enumerate(val_loader):

            inputs = inputs.to(device)
            s_inputs = s_inputs.to(device)
            target = target.to(device)

            # evaluate the model on the validation set
            out = model(inputs, s_inputs)

            out = torch.reshape(out, target.shape)
            out = out.to(device)

            # Calculate validation loss
            loss = criterion(out, target).detach()

            # predicted = torch.argmax(out.data, dim=1)

            # total_predictions += target.size(0)

            # correct_prediction += (predicted == torch.argmax(target, dim=1)).float().sum().item()

            running_loss += loss.item()#*inputs.size(0)


        running_loss /= len(val_loader)

        # print("Validation Loss = ", running_loss)
        # acc = (correct_prediction/total_predictions) * 100
        # print("Training accuracy = ", acc, "%")

        return running_loss

In [ ]:
epoch_start = 0
epoch_end = 50

Train_loss = []
Test_loss = []
Test_acc = []
Train_acc = []

trial = "_64_"+str(Tx)+"_"+str(Ty)+"_50_epochs_new_setcn" + "_no_sq_net_no_embed" # + "_6_TCN"
PATH = "/content/gdrive/MyDrive/KFUPM/SolarRadiationPrediction/model" + '/'+'trial'+str(trial)+'_v.pth'
val_loss_start = 1000
train_loss_start = 1000

divs = epoch_end/10

for epoch in range(epoch_start, epoch_end):

    # print("Epoch Number = ", epoch)
    # Train
    training_loss, train_time = train_model(train_loader, model_tcn, loss_fn, optimizer)
    val_loss = evaluate_model(val_loader, model_tcn, loss_fn)

    Train_loss.append(training_loss)
    Test_loss.append(val_loss)
    # Test_acc.append(val_acc)
    # Train_acc.append(taining_acc)

    # if training_loss < train_loss_start:
    #     PATH = "/content/gdrive/MyDrive/KFUPM/SolarRadiationPrediction/model" + '/'+'trial'+str(trial)+'_t.pth'
    #     torch.save(model_tcn.state_dict(), PATH)
    #     train_loss_start = training_loss

    if val_loss < val_loss_start:
        PATH = "/content/gdrive/MyDrive/KFUPM/SolarRadiationPrediction/model" + '/'+'trial'+str(trial)+'_v.pth'
        torch.save(model_tcn.state_dict(), PATH)
        val_loss_start = val_loss
    if np.isnan(val_loss):
        break

    # Print log of accuracy and loss
    scheduler.step(val_loss)
    if (epoch+1)%divs == 0:
        print("Epoch: "+str(epoch+1)+", Training loss: "+str(training_loss)+", Validation loss:"+str(val_loss))
        print(optimizer.param_groups[0]['lr'])
        print(scheduler._last_lr)
        print("="*20, "| saved training loss = ", train_loss_start, "| saved val loss = ", val_loss_start)

dictionary = {'train_loss': Train_loss, 'val_loss': val_loss}
with open("/content/gdrive/MyDrive/KFUPM/SolarRadiationPrediction/model" + '/saved_dictionary'+str(trial)+'.pkl', 'wb') as f:
    pickle.dump(dictionary, f)
print(trial)

78.6906555891037
76.76931056380272
77.36727192997932
76.44691696763039
76.92671602964401
Epoch: 5, Training loss: 0.377091745243353, Validation loss:0.34862159768288786
0.0001
[0.0001]
==================== | saved training loss =  1000 | saved val loss =  0.34862159768288786
75.07433560490608
73.31882634758949
69.6853855252266
66.33136716485023
63.929758340120316
Epoch: 10, Training loss: 0.3133811683339231, Validation loss:0.2820355539972132
0.0001
[0.0001]
==================== | saved training loss =  1000 | saved val loss =  0.2820355539972132
62.08582204580307
60.41835430264473
58.63434845209122
56.62721464037895
54.88361403346062
Epoch: 15, Training loss: 0.2690373236934344, Validation loss:0.24029516191645103
0.0001
[0.0001]
==================== | saved training loss =  1000 | saved val loss =  0.24029516191645103
53.046025574207306
50.55676743388176
47.89592890441418
43.43929052352905
37.69311021268368
Epoch: 20, Training loss: 0.18477014810139059, Validation loss:0.231263640252

In [ ]:
my_path = PATH.split('/')[-1]
print(my_path)

trial_64_48_60_50_epochs_new_setcn_no_sq_net_no_embed_v.pth


In [ ]:
# 0.006810544355010445
#0.015442172915209084
#5.880583208054304
#saved training loss =  0.004708505834319762 | saved val loss =  0.005296421283856034
#saved training loss =  0.004578455781134276 | saved val loss =  0.005215046340890694
#saved training loss =  0.004374954263567106 | saved val loss =  0.005082385327841621
#saved training loss =  0.004468559845258574 | saved val loss =  0.005138992093270645
#saved training loss =  0.004533737260786878 | saved val loss =  0.005202901383745484
# 0.023844994325190782
model_tcn.load_state_dict(torch.load("/content/gdrive/MyDrive/KFUPM/SolarRadiationPrediction/model" + '/' + str(my_path)
))# , map_location=torch.device('cpu') '/trial_128_48_24_setcn_v.pth'

device = torch.device("cuda" if cuda else "cpu")
model_tcn.to(device)
print("done")
# print(model)

done


In [ ]:
class DatasetTest(Dataset):

    # load the dataset
    def __init__(self, x_data):

        # store the inputs and outputs
        self.X = x_data
        self.S = x_data.transpose(0, 2, 1)

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, index):

        ### Get data at index pair with context (1 line)
        xx = torch.from_numpy(self.X[index]).float()
        s_xx = torch.from_numpy(self.S[index]).float()
        return xx, s_xx

In [ ]:
test_data = DatasetTest(x_test)
test_args = dict(shuffle = False, batch_size = 16,
                  num_workers = num_workers, pin_memory = True) if cuda else dict(shuffle = False, batch_size = 1)
test_loader = DataLoader(test_data, **test_args)

In [ ]:
def final_t_model(test_loader, model):

    print("Testing...")

    with torch.no_grad():


        p = []
        # Set model in validation mode
        model.eval()

        for i, (inputs, s_inputs) in enumerate(test_loader):

            inputs = inputs.to(device)
            s_inputs = s_inputs.to(device)

            # evaluate the model on the validation set
            out = model(inputs, s_inputs)

            # predicted = torch.argmax(out.data, dim=1)

            p = p + (out.cpu().numpy()).tolist()

        return p

In [ ]:
myP = final_t_model(test_loader, model_tcn)
# myP_np = ((np.asarray(myP))).reshape(-1, 1)
# myP_trans = scalar_y.inverse_transform(myP_np)
# s = np.asarray(myP)
# myP_trans = (s *max(data_copy_1['GSR']))y_test_original
print("Done")

Testing...
Done


In [ ]:
myP_np = ((np.asarray(myP))) # .reshape(-1, 1)
myP_trans = scalar_y.inverse_transform(myP_np)
myP_trans.shape

In [ ]:
np.max(np.max(myP_trans, axis=0), axis=0) # 1031.39179397, 1047.39752078, 1045.75067592, 1028.77295187

In [ ]:
counts, bins = np.histogram(y_test_original.flatten())
plt.hist(bins[:-1], bins, weights=counts)

In [ ]:
counts, bins = np.histogram(myP_trans.flatten())
plt.hist(bins[:-1], bins, weights=counts)

In [ ]:
replace_negatives = np.vectorize(lambda x: 0 if x < 0 else x)
myP_trans_fixed = replace_negatives(myP_trans)

In [ ]:
x = np.arange(len(y_test_original[30]))

index__ = 44

plt.figure()
plt.plot(x, myP_trans[index__], label ='prediction')
plt.plot(x, y_test_original[index__], '-.', label ='true value')

plt.legend()
plt.xlabel("X-axis data")
plt.ylabel("Y-axis data")
plt.title('multiple plots')
plt.show()

In [ ]:
def compare_df(index):
    my_data = {'true_value':y_test_original[index], 'pred_tcn':myP_trans[index]}
    df_to_plot = pd.DataFrame(data = my_data, index = test_df.index[index:index+Ty])
    return df_to_plot

out_df = compare_df(30)
# out_df.to_csv('/content/gdrive/MyDrive/KFUPM/SolarRadiationPrediction/outputs_data/' + 'gsr_pred_combined_12.csv')

In [ ]:
out_df

,true_value,pred_tcn
TIMESTAMP,,
2019-04-15 05:00:00,0.000,197.774671
2019-04-15 06:00:00,31.260,144.320211
2019-04-15 07:00:00,240.700,176.348255
2019-04-15 08:00:00,498.100,260.029382
2019-04-15 09:00:00,723.900,422.306560
2019-04-15 10:00:00,895.000,582.013764
2019-04-15 11:00:00,1014.000,734.714824
2019-04-15 12:00:00,1066.000,903.716431
2019-04-15 13:00:00,1039.000,954.454710


In [ ]:
# out_df.to_csv('/content/gdrive/MyDrive/KFUPM/SolarRadiationPrediction/outputs_data/' + 'gsr_pred_combined.csv')

In [ ]:
y_true, y_pred = y_test_original, myP_trans # myP_trans myP_trans_fixed
mae = mean_absolute_error(y_true, y_pred)
r2s = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)

In [ ]:
print("zero removed 64 TCN with "+str(Tx)+" lagged hours "+str(Ty)+" next sequence")
print("Mean Squared Error =", mse)
print("Mean Absolute Error =", mae)
print("Root Mean Squared Error =", rmse)
print("R2 Score =", r2s)

zero removed 64 TCN with 48 lagged hours 60 next sequence
Mean Squared Error = 43429.255502760985
Mean Absolute Error = 153.46014748038377
Root Mean Squared Error = 201.30582986676595
R2 Score = 0.6104127864924072


In [ ]:
#****************************************************************************
# STATCN - Embedding + TCN + LSTM - SE

# zero removed 64 TCN with 12 lagged hours 1 next sequence
# Mean Squared Error = 1151.1589720245904
# Mean Absolute Error = 13.369176982110753
# Root Mean Squared Error = 33.928733722680995
# R2 Score = 0.9896702573533166

# zero removed 64 TCN with 24 lagged hours 1 next sequence
# Mean Squared Error = 1172.2077063335755
# Mean Absolute Error = 13.811356621049482
# Root Mean Squared Error = 34.23751898624629
# R2 Score = 0.9894832650446661

# zero removed 64 TCN with 48 lagged hours 1 next sequence
# Mean Squared Error = 1535.6070523486949
# Mean Absolute Error = 14.64266927826375
# Root Mean Squared Error = 39.186822432403154
# R2 Score = 0.9861969745131596

# zero removed 64 TCN with 72 lagged hours 1 next sequence
# Mean Squared Error = 1973.5906151379177
# Mean Absolute Error = 15.276717730258
# Root Mean Squared Error = 44.42511243810102
# R2 Score = 0.9822658787014605

# zero removed 64 TCN with 12 lagged hours 6 next sequence
# Mean Squared Error = 2646.502011882249
# Mean Absolute Error = 22.50556299794324
# Root Mean Squared Error = 51.04425674147685
# R2 Score = 0.9762529245468924

# zero removed 64 TCN with 24 lagged hours 6 next sequence
# Mean Squared Error = 3009.526335033876
# Mean Absolute Error = 25.817021207152823
# Root Mean Squared Error = 54.03736125919883
# R2 Score = 0.9729977199214869

# zero removed 64 TCN with 48 lagged hours 6 next sequence
# Mean Squared Error = 3753.0739609376237
# Mean Absolute Error = 23.206962549040558
# Root Mean Squared Error = 60.52612836410119
# R2 Score = 0.9662722058244192

# zero removed 64 TCN with 72 lagged hours 6 next sequence
# Mean Squared Error = 4382.6922462553475
# Mean Absolute Error = 24.333815724839564
# Root Mean Squared Error = 65.38030310627317
# R2 Score = 0.9606197565943457

# zero removed 64 TCN with 12 lagged hours 12 next sequence
# Mean Squared Error = 3582.632749784923
# Mean Absolute Error = 26.6169393604542
# Root Mean Squared Error = 59.415773856646716
# R2 Score = 0.9678564651597671

# zero removed 64 TCN with 24 lagged hours 12 next sequence
# Mean Squared Error = 4203.0097551870085
# Mean Absolute Error = 31.327127054593316
# Root Mean Squared Error = 64.61247355348793
# R2 Score = 0.9622870445055196

# zero removed 64 TCN with 12 lagged hours 24 next sequence
# Mean Squared Error = 4838.423112902247
# Mean Absolute Error = 32.245393418392666
# Root Mean Squared Error = 69.34542301867516
# R2 Score = 0.9565915572900682

# zero removed 64 TCN with 24 lagged hours 24 next sequence
# Mean Squared Error = 5094.2999331427545
# Mean Absolute Error = 32.20365011396038
# Root Mean Squared Error = 70.89143938869898
# R2 Score = 0.954286089530546

# zero removed 64 TCN with 48 lagged hours 24 next sequence
# Mean Squared Error = 6085.457967394887
# Mean Absolute Error = 35.037328175073526
# Root Mean Squared Error = 77.61375137456689
# R2 Score = 0.9453704906899244

# zero removed 64 TCN with 12 lagged hours 36 next sequence
# Mean Squared Error = 6054.260501565556
# Mean Absolute Error = 36.174502303721596
# Root Mean Squared Error = 77.10389827337414
# R2 Score = 0.9456943535503071

# zero removed 64 TCN with 24 lagged hours 36 next sequence
# Mean Squared Error = 7490.996404258641
# Mean Absolute Error = 46.46493214254009
# Root Mean Squared Error = 85.69510278691864
# R2 Score = 0.9327855273506422

# zero removed 64 TCN with 48 lagged hours 36 next sequence
# Mean Squared Error = 8215.013073080569
# Mean Absolute Error = 48.52877647403406
# Root Mean Squared Error = 90.22125569730882
# R2 Score = 0.9262702562127468

# zero removed 64 TCN with 12 lagged hours 48 next sequence
# Mean Squared Error = 6276.106586498662
# Mean Absolute Error = 33.59529505731221
# Root Mean Squared Error = 78.64399021469904
# R2 Score = 0.9437156685803995

# zero removed 64 TCN with 24 lagged hours 48 next sequence
# Mean Squared Error = 6778.165804887644
# Mean Absolute Error = 38.113673686878634
# Root Mean Squared Error = 81.7247048415947
# R2 Score = 0.9392044653837077

# zero removed 64 TCN with 48 lagged hours 48 next sequence
# Mean Squared Error = 10799.35158139565
# Mean Absolute Error = 58.303386532313596
# Root Mean Squared Error = 103.18730853060454
# R2 Score = 0.9031050495079139

# zero removed 64 TCN with 12 lagged hours 60 next sequence
# Mean Squared Error = 11064.893567337596
# Mean Absolute Error = 64.26394882448693
# Root Mean Squared Error = 103.8804293981405
# R2 Score = 0.9007790084127177

# zero removed 64 TCN with 24 lagged hours 60 next sequence
# Mean Squared Error = 9129.642324530729
# Mean Absolute Error = 50.24755875229687
# Root Mean Squared Error = 94.04561530346726
# R2 Score = 0.9181190951499902

# zero removed 64 TCN with 48 lagged hours 60 next sequence
# Mean Squared Error = 43429.255502760985
# Mean Absolute Error = 153.46014748038377
# Root Mean Squared Error = 201.30582986676595
# R2 Score = 0.6104127864924072

In [ ]:
#****************************************************************************
# STATCN + Embedding + TCN + LSTM - SE

# zero removed 64 TCN with 12 lagged hours 1 next sequence
# Mean Squared Error = 1101.8723447728153
# Mean Absolute Error = 13.534562913431047
# Root Mean Squared Error = 33.194462561891484
# R2 Score = 0.9901125230940235

# zero removed 64 TCN with 24 lagged hours 1 next sequence
# Mean Squared Error = 1167.4615878301754
# Mean Absolute Error = 13.48896521168433
# Root Mean Squared Error = 34.16813702603897
# R2 Score = 0.9895258459542584

# zero removed 64 TCN with 48 lagged hours 1 next sequence
# Mean Squared Error = 1419.4199765865853
# Mean Absolute Error = 13.40943138355308
# Root Mean Squared Error = 37.675190465166665
# R2 Score = 0.9872413388025348

# zero removed 64 TCN with 72 lagged hours 1 next sequence
# Mean Squared Error = 1800.116342474684
# Mean Absolute Error = 14.422099644853482
# Root Mean Squared Error = 42.427777958251404
# R2 Score = 0.9838246689439702

# zero removed 64 TCN with 12 lagged hours 6 next sequence
# Mean Squared Error = 2715.93120335618
# Mean Absolute Error = 24.060894110322593
# Root Mean Squared Error = 51.56598913923182
# R2 Score = 0.9756299379109429

# zero removed 64 TCN with 24 lagged hours 6 next sequence
# Mean Squared Error = 2689.244018582216
# Mean Absolute Error = 20.38600650657416
# Root Mean Squared Error = 51.15724483337734
# R2 Score = 0.9758714425859999

# zero removed 64 TCN with 48 lagged hours 6 next sequence
# Mean Squared Error = 3616.2507662506155
# Mean Absolute Error = 22.348232579870956
# Root Mean Squared Error = 59.41551301586758
# R2 Score = 0.9675017879585396

# zero removed 64 TCN with 72 lagged hours 6 next sequence
# Mean Squared Error = 4516.468820124294
# Mean Absolute Error = 26.932631864556527
# Root Mean Squared Error = 66.99009701854287
# R2 Score = 0.9594177205523593

# zero removed 64 TCN with 12 lagged hours 12 next sequence
# Mean Squared Error = 3577.6585103450834
# Mean Absolute Error = 26.292257604598948
# Root Mean Squared Error = 59.29049920951649
# R2 Score = 0.9679010744808708

# zero removed 64 TCN with 24 lagged hours 12 next sequence
# Mean Squared Error = 3812.079935152042
# Mean Absolute Error = 24.281641989557656
# Root Mean Squared Error = 61.19671192669043
# R2 Score = 0.9657944754454308

# zero removed 64 TCN with 48 lagged hours 12 next sequence
# Mean Squared Error = 4796.187051328752
# Mean Absolute Error = 27.60279234233161
# Root Mean Squared Error = 68.81295155957069
# R2 Score = 0.9569205559819198

# zero removed 64 TCN with 72 lagged hours 12 next sequence
# Mean Squared Error = 5575.595868366807
# Mean Absolute Error = 31.022843712536666
# Root Mean Squared Error = 74.34181687657879
# R2 Score = 0.9499053573507238

# zero removed 64 TCN with 12 lagged hours 24 next sequence
# Mean Squared Error = 5799.9063845730125
# Mean Absolute Error = 38.562806782123026
# Root Mean Squared Error = 75.35113677378983
# R2 Score = 0.9479647354986063

# zero removed 64 TCN with 24 lagged hours 24 next sequence
# Mean Squared Error = 4713.410149188508
# Mean Absolute Error = 28.92233816142038
# Root Mean Squared Error = 68.20239058532947
# R2 Score = 0.9577044451537828

# zero removed 64 TCN with 48 lagged hours 24 next sequence
# Mean Squared Error = 6426.147234037572
# Mean Absolute Error = 34.8193299784488
# Root Mean Squared Error = 79.9632463880872
# R2 Score = 0.9423147253131011

# zero removed 64 TCN with 72 lagged hours 24 next sequence
# Mean Squared Error = 7118.6962594183615
# Mean Absolute Error = 33.33877839999385
# Root Mean Squared Error = 84.21008680206094
# R2 Score = 0.9360752845852452

# zero removed 64 TCN with 12 lagged hours 36 next sequence
# Mean Squared Error = 6638.900159679032
# Mean Absolute Error = 40.37228776017547
# Root Mean Squared Error = 80.80385342002843
# R2 Score = 0.9404497220409043

# zero removed 64 TCN with 24 lagged hours 36 next sequence
# Mean Squared Error = 6448.772684395712
# Mean Absolute Error = 33.90095552880862
# Root Mean Squared Error = 79.8126507417264
# R2 Score = 0.9421405809195009

# zero removed 64 TCN with 48 lagged hours 36 next sequence
# Mean Squared Error = 7621.234881910824
# Mean Absolute Error = 41.063846613907145
# Root Mean Squared Error = 86.84170217449288
# R2 Score = 0.9315985363513976

# zero removed 64 TCN with 72 lagged hours 36 next sequence
# Mean Squared Error = 8269.866830357583
# Mean Absolute Error = 38.00379393874196
# Root Mean Squared Error = 90.53131085442384
# R2 Score = 0.9257683268390396

# zero removed 64 TCN with 12 lagged hours 48 next sequence
# Mean Squared Error = 6788.860916127417
# Mean Absolute Error = 39.45773952799775
# Root Mean Squared Error = 81.78809124879918
# R2 Score = 0.9391177595512552

# zero removed 64 TCN with 24 lagged hours 48 next sequence
# Mean Squared Error = 6716.974585070769
# Mean Absolute Error = 36.86447599857622
# Root Mean Squared Error = 81.40611799047483
# R2 Score = 0.9397543893840781

# zero removed 64 TCN with 48 lagged hours 48 next sequence
# Mean Squared Error = 7785.601162044707
# Mean Absolute Error = 38.620771101432375
# Root Mean Squared Error = 87.74623020665332
# R2 Score = 0.9301466728492754

# zero removed 64 TCN with 72 lagged hours 48 next sequence
# Mean Squared Error = 9234.051338928379
# Mean Absolute Error = 42.19939930510122
# Root Mean Squared Error = 95.7151991677437
# R2 Score = 0.9171304863558838

# zero removed 64 TCN with 12 lagged hours 60 next sequence
# Mean Squared Error = 8617.992076272549
# Mean Absolute Error = 49.97224729138535
# Root Mean Squared Error = 91.92834114547775
# R2 Score = 0.922728339483516

# zero removed 64 TCN with 24 lagged hours 60 next sequence
# Mean Squared Error = 102424.55901181143
# Mean Absolute Error = 267.02491626672037
# Root Mean Squared Error = 316.26729183365836
# R2 Score = 0.08171832862514451

In [ ]:
#****************************************************************************
# STATCN + Embedding + TCN + SE + LSTM
#
# zero removed 64 TCN with 12 lagged hours 1 next sequence
# Mean Squared Error = 1099.2518354655936
# Mean Absolute Error = 12.909539274262693
# Root Mean Squared Error = 33.15496698031222
# R2 Score = 0.9901360378191003

# zero removed 64 TCN with 24 lagged hours 1 next sequence
# Mean Squared Error = 1217.2390158571454
# Mean Absolute Error = 12.521310259285636
# Root Mean Squared Error = 34.88895263342174
# R2 Score = 0.989079256143861

# zero removed 64 TCN with 48 lagged hours 1 next sequence
# Mean Squared Error = 1494.359904506252
# Mean Absolute Error = 13.417004046198064
# Root Mean Squared Error = 38.65695156768381
# R2 Score = 0.9865677304510526

# zero removed 64 TCN with 72 lagged hours 1 next sequence
# Mean Squared Error = 1848.8262278818104
# Mean Absolute Error = 14.543446547068477
# Root Mean Squared Error = 42.997979346497324
# R2 Score = 0.9833869758329359

# zero removed 64 TCN with 12 lagged hours 6 next sequence
# Mean Squared Error = 2500.8206696750185
# Mean Absolute Error = 21.50150384104997
# Root Mean Squared Error = 49.430216503897675
# R2 Score = 0.9775601266815452

# zero removed 64 TCN with 24 lagged hours 6 next sequence
# Mean Squared Error = 2824.159980278311
# Mean Absolute Error = 21.47760496455913
# Root Mean Squared Error = 52.63594197370455
# R2 Score = 0.9746609235379516

# zero removed 64 TCN with 48 lagged hours 6 next sequence
# Mean Squared Error = 3589.4017174851365
# Mean Absolute Error = 24.170729179442294
# Root Mean Squared Error = 59.38510947700499
# R2 Score = 0.9677434960176421

# zero removed 64 TCN with 72 lagged hours 6 next sequence
# Mean Squared Error = 4156.518385033188
# Mean Absolute Error = 24.548213171634966
# Root Mean Squared Error = 64.0403889717442
# R2 Score = 0.9626520192325759

# zero removed 64 TCN with 12 lagged hours 12 next sequence
# Mean Squared Error = 3665.9747782794097
# Mean Absolute Error = 26.801550724299005
# Root Mean Squared Error = 60.07893755654451
# R2 Score = 0.9671086933138944

# zero removed 64 TCN with 24 lagged hours 12 next sequence
# Mean Squared Error = 4054.9037475061596
# Mean Absolute Error = 27.291402248544102
# Root Mean Squared Error = 63.4607507380554
# R2 Score = 0.9636159839862412

# zero removed 64 TCN with 48 lagged hours 12 next sequence
# Mean Squared Error = 4789.063456988325
# Mean Absolute Error = 27.542716654982765
# Root Mean Squared Error = 68.90663512599484
# R2 Score = 0.9569856540444649

# zero removed 64 TCN with 72 lagged hours 12 next sequence
# Mean Squared Error = 5365.039258859423
# Mean Absolute Error = 31.208133247507874
# Root Mean Squared Error = 72.94878932797336
# R2 Score = 0.951797107473447

# zero removed 64 TCN with 12 lagged hours 24 next sequence
# Mean Squared Error = 4935.107448821745
# Mean Absolute Error = 33.13642478446533
# Root Mean Squared Error = 70.05550124214639
# R2 Score = 0.9557241103516091

# zero removed 64 TCN with 12 lagged hours 24 next sequence 100 epochs
# Mean Squared Error = 4592.586310797086
# Mean Absolute Error = 29.532616678106212
# Root Mean Squared Error = 67.52234496185353
# R2 Score = 0.9587969070711312

# zero removed 64 TCN with 24 lagged hours 24 next sequence 100 epochs
# Mean Squared Error = 4941.9864894987495
# Mean Absolute Error = 31.0192525596705
# Root Mean Squared Error = 69.8379891174477
# R2 Score = 0.9556533327795901

# zero removed 64 TCN with 48 lagged hours 24 next sequence
# Mean Squared Error = 6560.019078920879
# Mean Absolute Error = 37.8387828867374
# Root Mean Squared Error = 80.75087952775424
# R2 Score = 0.9411124482827379

# zero removed 64 TCN with 72 lagged hours 24 next sequence
# Mean Squared Error = 6398.37641517766
# Mean Absolute Error = 33.11053918615869
# Root Mean Squared Error = 79.68630336163244
# R2 Score = 0.9425415751591505

# zero removed 64 TCN with 12 lagged hours 36 next sequence
# Mean Squared Error = 6004.878042900557
# Mean Absolute Error = 36.645262909035154
# Root Mean Squared Error = 76.97752769856511
# R2 Score = 0.9461395215439915

# zero removed 64 TCN with 24 lagged hours 36 next sequence
# Mean Squared Error = 8976.401451946616
# Mean Absolute Error = 53.788019246851896
# Root Mean Squared Error = 94.04781107564624
# R2 Score = 0.9194602963947514

# zero removed 64 TCN with 48 lagged hours 36 next sequence
# Mean Squared Error = 14372.811656889264
# Mean Absolute Error = 67.12395744542502
# Root Mean Squared Error = 116.96869173826377
# R2 Score = 0.8709850142832555

# zero removed 64 TCN with 72 lagged hours 36 next sequence
# Mean Squared Error = 9083.64651967238
# Mean Absolute Error = 46.82343306016959
# Root Mean Squared Error = 94.72769188789522
# R2 Score = 0.9184621895023078

# zero removed 64 TCN with 12 lagged hours 48 next sequence
# Mean Squared Error = 7263.569954392103
# Mean Absolute Error = 41.40396436683874
# Root Mean Squared Error = 84.38357660734184
# R2 Score = 0.9348570658927189

# zero removed 64 TCN with 24 lagged hours 48 next sequence
# Mean Squared Error = 7842.586840291503
# Mean Absolute Error = 42.224060357002045
# Root Mean Squared Error = 87.57859192788392
# R2 Score = 0.9296527133745439

# zero removed 64 TCN with 48 lagged hours 48 next sequence
# Mean Squared Error = 8115.439963340369
# Mean Absolute Error = 42.309949813594905
# Root Mean Squared Error = 89.52085803740147
# R2 Score = 0.9271860394879964

# zero removed 64 TCN with 72 lagged hours 48 next sequence
# Mean Squared Error = 11318.246541096116
# Mean Absolute Error = 55.59811822361312
# Root Mean Squared Error = 105.91876507998983
# R2 Score = 0.8984242598308665

# zero removed 64 TCN with 12 lagged hours 60 next sequence
# Mean Squared Error = 7589.402887215682
# Mean Absolute Error = 39.53836398371147
# Root Mean Squared Error = 86.28273607714007
# R2 Score = 0.9319497943338677

# zero removed 64 TCN with 24 lagged hours 60 next sequence
# Mean Squared Error = 8985.661675233616
# Mean Absolute Error = 47.37288128875154
# Root Mean Squared Error = 93.2231980223577
# R2 Score = 0.9194079603591958

In [ ]:
# zero removed 64 TCN with 48 lagged hours 48 next sequence
# Mean Squared Error = 66835.04960571967
# Mean Absolute Error = 197.5112617951289
# Root Mean Squared Error = 241.4084695542823
# R2 Score = 0.3820687902505582

# zero removed 64 TCN with 24 lagged hours 48 next sequence
# Mean Squared Error = 9941.185969087315
# Mean Absolute Error = 56.70725641002911
# Root Mean Squared Error = 98.97660321445879
# R2 Score = 0.9081469717457548

# zero removed 64 TCN with 24 lagged hours 36 next sequence
# Mean Squared Error = 7003.566803677504
# Mean Absolute Error = 38.44810893597829
# Root Mean Squared Error = 83.14142290259855
# R2 Score = 0.9352783899616978

# zero removed 64 TCN with 48 lagged hours 36 next sequence
# Mean Squared Error = 8118.341974316891
# Mean Absolute Error = 37.365258710313945
# Root Mean Squared Error = 89.32479273286866
# R2 Score = 0.9249463166318335

# zero removed 64 TCN with 72 lagged hours 36 next sequence
# Mean Squared Error = 15988.937500037473
# Mean Absolute Error = 79.50744070359724
# Root Mean Squared Error = 125.26557971761463
# R2 Score = 0.852122000328151

# zero removed 64 TCN with 72 lagged hours 24 next sequence
# Mean Squared Error = 7686.356623817871
# Mean Absolute Error = 34.486188373248005
# Root Mean Squared Error = 87.30103381512076
# R2 Score = 0.9289010280274441

# zero removed 128 TCN with 72 lagged hours 24 next sequence
# Mean Squared Error = 7272.651548085596
# Mean Absolute Error = 32.326533688771484
# Root Mean Squared Error = 85.09060078967543
# R2 Score = 0.9327301802879985

# zero removed 128 TCN with 48 lagged hours 24 next sequence
# Mean Squared Error = 6336.600313793646
# Mean Absolute Error = 32.02449951474096
# Root Mean Squared Error = 79.25078728430293
# R2 Score = 0.9414066419513416

# zero removed 128 TCN with 24 lagged hours 24 next sequence
# Mean Squared Error = 12726.897303802418
# Mean Absolute Error = 69.42971594763931
# Root Mean Squared Error = 111.8249738770586
# R2 Score = 0.8823627085805725

# zero removed 128 TCN with 72 lagged hours 12 next sequence
# Mean Squared Error = 5944.047066252241
# Mean Absolute Error = 27.614704726825607
# Root Mean Squared Error = 76.53075349780593
# R2 Score = 0.9450037977758451

# zero removed 128 TCN with 48 lagged hours 12 next sequence
# Mean Squared Error = 4765.059555585541
# Mean Absolute Error = 26.654842791584624
# Root Mean Squared Error = 68.52075685487723
# R2 Score = 0.9559398454052318

# zero removed 64 TCN with 24 lagged hours 12 next sequence
# Mean Squared Error = 7431.544340239208
# Mean Absolute Error = 45.18900597279756
# Root Mean Squared Error = 84.01187224713897
# R2 Score = 0.9312625857308663

# zero removed 64 TCN with 72a lagged hours 6 next sequence
# Mean Squared Error = 4936.932561550985
# Mean Absolute Error = 26.393934317005925
# Root Mean Squared Error = 69.27771703707766
# R2 Score = 0.9542934262918242

# zero removed 64 TCN with 48a lagged hours 6 next sequence
# Mean Squared Error = 3832.0069919128023
# Mean Absolute Error = 24.557209463359982
# Root Mean Squared Error = 61.19944812871763
# R2 Score = 0.9645638988354402

# zero removed 64 TCN with 24a lagged hours 6 next sequence
# Mean Squared Error = 4401.94419496796
# Mean Absolute Error = 26.624740404260535
# Root Mean Squared Error = 65.84937171090051
# R2 Score = 0.9592808773503592

# 128 TCN with 24 lagged hours
# Mean Squared Error = 1021.4543658818118
# Mean Absolute Error = 11.47969818027391
# Root Mean Squared Error = 31.96019971592499
# R2 Score = 0.9905509018145809

# zero removed 128 TCN with 24 lagged hours
# Mean Squared Error = 1020.9886422174817
# Mean Absolute Error = 11.181218123634839
# Root Mean Squared Error = 31.95291289096319
# R2 Score = 0.9905552100526956

# zero removed 64 TCN with 24 lagged hours
# Mean Squared Error = 996.6256730750337
# Mean Absolute Error = 11.645388431334489
# Root Mean Squared Error = 31.56937872488202
# R2 Score = 0.9907805829085027

# zero removed 64 TCN RNN with 48 lagged hours
# Mean Squared Error = 1472.787996166696
# Mean Absolute Error = 13.664473609650795
# Root Mean Squared Error = 38.37692009745827
# R2 Score = 0.9863800300746124

# zero removed 64 TCN Added STA with 48 lagged hours
# Mean Squared Error = 1446.9611877310576
# Mean Absolute Error = 12.870758821942607
# Root Mean Squared Error = 38.038943041717886
# R2 Score = 0.9866188698499756

# zero removed 64 TCN with 72 lagged hours
# Mean Squared Error = 1983.2703380929806
# Mean Absolute Error = 14.949228286736764
# Root Mean Squared Error = 44.533923452722874
# R2 Score = 0.9816355552495802

# zero removed 64 TCN with 24a lagged hours
# Mean Squared Error = 974.9739374474608
# Mean Absolute Error = 11.588196637662904
# Root Mean Squared Error = 31.224572654360873
# R2 Score = 0.9909808751414827

In [ ]:
test_df.shape

(9948, 330)

In [ ]:
test_df.index[0:25]

DatetimeIndex(['2019-06-08 15:00:00', '2019-06-08 16:00:00',
               '2019-06-08 17:00:00', '2019-06-08 18:00:00',
               '2019-06-08 19:00:00', '2019-06-08 20:00:00',
               '2019-06-08 21:00:00', '2019-06-08 22:00:00',
               '2019-06-08 23:00:00', '2019-06-09 00:00:00',
               '2019-06-09 01:00:00', '2019-06-09 02:00:00',
               '2019-06-09 03:00:00', '2019-06-09 04:00:00',
               '2019-06-09 05:00:00', '2019-06-09 06:00:00',
               '2019-06-09 07:00:00', '2019-06-09 08:00:00',
               '2019-06-09 09:00:00', '2019-06-09 10:00:00',
               '2019-06-09 11:00:00', '2019-06-09 12:00:00',
               '2019-06-09 13:00:00', '2019-06-09 14:00:00',
               '2019-06-09 15:00:00'],
              dtype='datetime64[ns]', name='TIMESTAMP', freq=None)

In [ ]:
new_df = pd.DataFrame(data = myP_trans, index = test_df.index)
new_df.head(10)

,0,1,2,3,4,5
TIMESTAMP,,,,,,
2019-06-08 20:00:00,7.844639,-0.819948,1.023827,1.910344,0.655967,0.747971
2019-06-08 21:00:00,1.677327,0.066766,1.408102,0.527344,0.974400,1.564805
2019-06-08 22:00:00,-1.173785,-0.324351,0.602561,0.675390,1.426922,1.887010
2019-06-08 23:00:00,-2.906978,-0.768482,1.682203,0.640482,0.930481,1.470987
2019-06-09 00:00:00,-4.937375,-1.549238,1.952833,1.299484,1.621419,2.250098
2019-06-09 01:00:00,-7.259619,-3.475191,1.755238,4.176535,13.599663,53.227318
2019-06-09 02:00:00,-7.915248,-2.917411,5.447480,27.914227,124.024272,283.523414
2019-06-09 03:00:00,-9.826918,-5.681671,8.770519,99.549395,291.686397,480.512423
2019-06-09 04:00:00,-9.504105,2.578751,67.421324,259.072140,479.021044,649.655200


In [ ]:
all_days = pd.date_range(new_df.index.min(), new_df.index.max(), freq='H')

In [ ]:
out_df = new_df.reindex(all_days, fill_value=0)

13727

In [ ]:
# y_true, y_pred = y_test_original, myP_trans
# mae = mean_absolute_error(y_true, y_pred)
# r2s = r2_score(y_true, y_pred)
# mse = mean_squared_error(y_true, y_pred)
# rmse = mean_squared_error(y_true, y_pred, squared=False)

In [ ]:
# print("128 TCN with 24 lagged hours")
# print("Mean Squared Error =", mse)
# print("Mean Absolute Error =", mae)
# print("Root Mean Squared Error =", rmse)
# print("R2 Score =", r2s)

128 TCN with 24 lagged hours
Mean Squared Error = 1021.4543658818118
Mean Absolute Error = 11.47969818027391
Root Mean Squared Error = 31.96019971592499
R2 Score = 0.9905509018145809


In [ ]:
output_file = '/content/gdrive/MyDrive/KFUPM/SolarRadiationPrediction/outputs_data/' + 'gsr_pred_new.csv'

In [ ]:
d = {'original_output': y_test_original.flatten(), 'predicted_24_setcn': myP_trans.flatten()}
df = pd.DataFrame(data=d)
# df.to_csv('/content/gdrive/MyDrive/KFUPM/SolarRadiationPrediction/outputs_data/' + 'gsr_pred_new.csv', index = False)

In [ ]:
if os.path.exists(output_file):
    output_pd = read_csv(output_file)
    print("FIle exists")

In [ ]:
output_pd[trial] = myP_trans_fixed.flatten()

,original_output,predicted_24_setcn
0,0.0,-0.128771
1,0.0,-0.981355
2,0.0,-0.657545
3,0.0,-0.303178
4,0.0,-0.091248
...,...,...
9944,0.0,-1.059868
9945,0.0,-1.005593
9946,0.0,-1.265910
9947,0.0,-1.233513


In [ ]:
def plot_predictions(df_result):
    data = []

    value = go.Scatter(
        x=df_result.index,
        y=df_result.original_output,
        mode="lines",
        name="values",
        marker=dict(),
        text=df_result.index,
        line=dict(color="rgba(0,0,255, 0.3)"),
    )
    data.append(value)

    # baseline = go.Scatter(
    #     x=df_baseline.index,
    #     y=df_baseline.prediction,
    #     mode="lines",
    #     line={"dash": "dot"},
    #     name='linear regression',
    #     marker=dict(),
    #     text=df_baseline.index,
    #     opacity=0.8,
    # )
    # data.append(baseline)

    prediction = go.Scatter(
        x=df_result.index,
        y=df_result.predicted_24_setcn,
        mode="lines",
        line={"dash": "dot"},
        name='predictions',
        marker=dict(),
        text=df_result.index,
        opacity=0.8,
    )
    data.append(prediction)

    layout = dict(
        title="Predictions vs Actual GSR for the dataset",
        xaxis=dict(title="Time", ticklen=5, zeroline=False),
        yaxis=dict(title="GSR", ticklen=5, zeroline=False),
    )

    fig = dict(data=data, layout=layout)
    iplot(fig)


# Set notebook mode to work in offline
# pyo.init_notebook_mode()

In [ ]:
plot_predictions(df)